In [1]:
import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 50,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'binary'
)

wx = w @ x.T
# [i for i in zip(wx.T,wx_outBits.T)]
# wx.T[0], q.saturating_clip(wx.T[0],inBits = 6,outBits = 4)
# q.get_array_bits(res)
# px.histogram(x=res.flatten(),title=f'{res.mean()}')

a = wx_outBits.T[::].T
a

array([[ 0,  1, -1, ...,  2, -1,  3],
       [ 1,  1,  0, ...,  2,  3,  1],
       [ 0,  0, -1, ..., -3, -5, -2],
       ...,
       [-1, -3,  1, ..., -2, -2,  0],
       [ 0, -4, -3, ...,  0,  1, -1],
       [-2,  0, -1, ...,  2,  2, -1]])

In [2]:
wx_outBits.T[::-1].T

array([[ 3, -1,  2, ..., -1,  1,  0],
       [ 1,  3,  2, ...,  0,  1,  1],
       [-2, -5, -3, ..., -1,  0,  0],
       ...,
       [ 0, -2, -2, ...,  1, -3, -1],
       [-1,  1,  0, ..., -3, -4,  0],
       [-1,  2,  2, ..., -1,  0, -2]])

In [3]:
[np.dot(w[i],x[0]) for i in range(w.shape[0])]

[3,
 5,
 -3,
 2,
 -2,
 9,
 4,
 0,
 -3,
 -1,
 3,
 2,
 8,
 13,
 8,
 1,
 9,
 5,
 1,
 -1,
 5,
 -5,
 4,
 5,
 -1,
 0,
 1,
 2,
 3,
 9,
 -4,
 12]

In [4]:
(w @ x.T).T

array([[  3,   5,  -3, ...,   9,  -4,  12],
       [  4,   4,   0, ...,  11,  12,   6],
       [  0,   0,  -1, ...,  -9, -17,  -7],
       ...,
       [ -3, -10,   4, ...,  -6,  -5,   3],
       [  1, -14,  -9, ...,   2,   4,  -1],
       [ -6,   1,  -2, ...,   8,   9,  -2]])

In [10]:
w.T.shape

(128, 32)